### imports

In [ ]:
import numpy as np 
import pandas as pd 

### simulator

A desired SDE can be simulated through the following equation: 

$dX_t = f(X_t)dt + g(X_t)dW_t$, where these inputs are the following 

- $f(X_t)$ is the custom drift or deterministic baseline function
- $g(X_t)$ is the diffusion (stochastic part) that scales randomness factor
- $dW_t$ the wiener process (brownian motion)

We will use these as inputs to our simulator, and different inputs will create different SDE

In [ ]:
def eu_simulator(f_t, g_t, W_t, x0, T, dt):
    """
    Simulate an SDE using the Euler-Maruyama method.
    
    Parameters:
    f_t: function - the drift function f(X_t)
    g_t: function - the diffusion function g(X_t)
    W_t: array-like - the Wiener process increments
    x0: float - initial condition
    T: float - total time to simulate
    dt: float - time step size
    
    Returns:
    X: array - simulated values of X_t at each time step
    """
    N = int(T / dt)  # number of time steps
    X = np.zeros(N + 1)
    X[0] = x0
    
    for i in range(1, N + 1):
        t = i * dt
        dW = W_t[i-1]  # Wiener increment for this time step
        X[i] = X[i-1] + f_t(X[i-1]) * dt + g_t(X[i-1]) * dW
        
    return X 


### simulating OU Process

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
